# AI Agent Core - 模型训练

本Notebook通过运行脚本训练模型。

## 1. 配置参数

In [ ]:
# ==================== 配置区 ====================
GITHUB_REPO = "https://github.com/hzt818/gameCube.git"
HF_TOKEN = ""  # 在这里填入你的HF token
HF_REPO_ID = "Xuchen818/openhzt"

BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"
NUM_EPOCHS = 3
BATCH_SIZE = 4
LEARNING_RATE = 2e-4
NUM_SAMPLES = 2000
OUTPUT_DIR = "/content/outputs"
# ==================== 配置结束 ====================

## 2. 环境准备

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets accelerate peft bitsandbytes trl huggingface_hub

## 3. 克隆项目

In [ ]:
!git clone {GITHUB_REPO} /content/gameCube

## 4. 生成训练数据

In [ ]:
!cd /content/gameCube && python scripts/generate_data.py --output data/train_data.json --num_samples {NUM_SAMPLES}

## 5. 训练模型 (训练+合并+测试)

In [ ]:
!cd /content/gameCube && python scripts/train.py \
    --mode all \
    --model_name {BASE_MODEL} \
    --output_dir {OUTPUT_DIR} \
    --num_epochs {NUM_EPOCHS} \
    --batch_size {BATCH_SIZE} \
    --learning_rate {LEARNING_RATE} \
    --num_samples {NUM_SAMPLES}

## 6. 上传到Hugging Face

In [ ]:
if HF_TOKEN:
    !cd /content/gameCube && python scripts/upload_hf.py \
        --model_path {OUTPUT_DIR}/merged_model \
        --repo_id {HF_REPO_ID} \
        --token {HF_TOKEN}
else:
    print("请先设置HF_TOKEN")

## 7. 下载模型

In [ ]:
import shutil
from google.colab import files

shutil.make_archive(f"{OUTPUT_DIR}/model", 'zip', f"{OUTPUT_DIR}/merged_model")
files.download(f"{OUTPUT_DIR}/model.zip")